In [1]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

### DH to W3 mapping merging with W3 gene_info 
+ attained alphabetical names from CF gene info 
+ (preparing to map to full mapping)


In [2]:
##load DH_W3 mapping and W3 gene_info 
DH_mapping = pd.read_table('./DH10B_and_W3110_gene_map.txt')
W3_info = pd.read_table('../Ecoli_K12_W3110_NC_007779_gene_info.txt')

DH_mapping.columns = ['DH', 'DH_names', 'W3', 'W3_names', 'drop']
#drop bad format of missing column value`
DH_mapping = DH_mapping[['DH', 'DH_names', 'W3', 'W3_names']].dropna(how='any')

In [3]:
#take unique W3
DH_mapped = DH_mapping.groupby(['W3_names']).agg(
    DH_names=pd.NamedAgg(column='DH_names', aggfunc=lambda x:','.join(x))).reset_index()

In [4]:
#merged with W3 gene_info 
W3_mapped = pd.merge(W3_info[['gbk_locus_tag','gbk_gene_id']], DH_mapped, left_on='gbk_locus_tag', right_on='W3_names')

In [5]:
W3_mapped.describe()

,gbk_locus_tag,gbk_gene_id,W3_names,DH_names
count,941,645,941,941
unique,941,642,941,869
top,Y75_RS08860,hypA,Y75_RS08860,ECDH10B_RS02890
freq,1,2,1,3


In [6]:
#Expand DH names that were combined and separated by ','
extra= W3_mapped['DH_names'].str.split(',', n=4, expand=True)
extra.columns = ['DH_name1','DH_name2','DH_name3','DH_name4' ]
#merged back with W3 gene alphabetical name
extra['W3_names'] = W3_mapped['W3_names']
extra['gene_id'] = W3_mapped['gbk_gene_id']

In [7]:
#collapsing the multiple DH names columns into extra rows
new = pd.melt(extra, id_vars=['W3_names','gene_id'], value_vars=['DH_name1', 'DH_name2', 'DH_name3', 'DH_name4'], 
       value_name='DH_names')

new = new[['W3_names', 'DH_names', 'gene_id']].dropna(subset=['DH_names'])

new.fillna('-', inplace=True)

In [8]:
# making unique DH name to prepare to map with full mapping
# combining multiple alphabetical names rows into one row seprated by commas 
W3_final = new.groupby(['DH_names']).agg(
    W3_names=pd.NamedAgg(column='W3_names', aggfunc=lambda x:','.join(x)), 
    gene_id=pd.NamedAgg(column='gene_id', aggfunc=lambda x:','.join(x))).reset_index()

In [9]:
# make seperate columns for the multiple gene names 
extra_gene_id = W3_final['gene_id'].str.split(',', n=3, expand=True)
extra_gene_id.columns = ['W3_name1', 'W3_name2', 'W3_name3']
W3_final = pd.concat([W3_final, extra_gene_id], axis=1)

In [10]:
#drop original gene_id columns that has multiple columns
W3_final.drop(['gene_id', 'W3_names'],axis=1, inplace=True)
W3_final = W3_final.replace('-',None)

In [11]:
W3_final.describe()

,DH_names,W3_name1,W3_name2,W3_name3
count,953,953,58,4
unique,953,593,55,4
top,ECDH10B_RS20390,pepD,sufB,quuD
freq,1,11,2,1


### ATCC and CF mapping merging with CF gene info 
+ attained alphabetical names from CF gene info 
+ (preparing to map to full mapping)

#### include both CF gene_info files

In [12]:
#include both CF mappings 
CF_info1 = pd.read_table('../Ecoli_CFT073_NC_004431_gene_info.txt')
CF_info2 = pd.read_table('../Ecoli_CFT073_AE014075_gene_info.txt')

CF_names1 = CF_info1[['gbk_gene_id', 'gbk_locus_tag', 'gbk_old_locus_tag']]
CF_names2 = CF_info2[['gbk_gene_id', 'gbk_locus_tag']]

In [13]:
CF_name = pd.merge(CF_names1, CF_names2, left_on='gbk_old_locus_tag', right_on='gbk_locus_tag', 
                  how='outer')
CF_name = CF_name[CF_name['gbk_locus_tag_x'].notna()] #only select valid C_RS locus tags 

CF_name['check'] = CF_name['gbk_gene_id_x'] == CF_name['gbk_gene_id_y'] #check if both gen_info provides the same info

## check in consistent naming
# print(len(CF_name))
# CF_name[(CF_name['gbk_gene_id_x'].notna()) & (CF_name['gbk_gene_id_y'].notna()) & (CF_name['check']==False)]

In [14]:
## Organizing
CF_name['CF_alt_name'] = np.where(CF_name['gbk_old_locus_tag'].isna(), CF_name['gbk_locus_tag_y'],
                                  CF_name['gbk_old_locus_tag'])
CF_name.rename(columns={'gbk_gene_id_x':'CF_name1', 'gbk_gene_id_y':'CF_name2', 
                       'gbk_locus_tag_x':'locus_tag'}, inplace=True)
CF_name = CF_name[['CF_name1', 'CF_name2', 'locus_tag', 'CF_alt_name']]

CF_name.describe()

,CF_name1,CF_name2,locus_tag,CF_alt_name
count,2855,3532,5024,4738
unique,2756,3532,5024,4738
top,tnpA,ydcO,C_RS19455,c3381
freq,19,1,1,1


#### ATCC CF mapping merging with both CF gene info 

In [15]:
## ATCC to CF ----------
#load data
ATCC_mapping = pd.read_table('./ATCC_and_CFT073_gene_map.txt')
ATCC_mapping.columns = ['ATCC', 'ATCC_names', 'CF', 'CF_names', 'drop']
ATCC_mapping = ATCC_mapping[['ATCC', 'ATCC_names',
                             'CF', 'CF_names']].drop_duplicates(keep='first').dropna(how='any').reset_index(drop=True)


In [16]:
#take unique CF
ATCC_mapped = ATCC_mapping.groupby(['CF_names']).agg(
    ATCC_names=pd.NamedAgg(column='ATCC_names', aggfunc=lambda x:','.join(x))).reset_index()

In [17]:
#merged with CF gene_info 
CF_mapped = pd.merge(CF_name, ATCC_mapped, left_on='locus_tag', right_on='CF_names')

CF_mapped.describe()

,CF_name1,CF_name2,locus_tag,CF_alt_name,CF_names,ATCC_names
count,598,716,940,912,940,940
unique,593,716,940,912,940,872
top,tnpA,ompR,C_RS19480,c1783,C_RS19480,DR76_RS00800
freq,3,1,1,1,1,4


In [18]:
#Expand ATCC names that were combined and separated by ','
extra= CF_mapped['ATCC_names'].str.split(',', n=4, expand=True)
extra.columns = ['ATCC_name1','ATCC_name2','ATCC_name3','ATCC_name4' ]
#merged back with CF gene alphabetical name
extra = pd.concat([CF_mapped.iloc[:, 0:4], extra], axis=1)

In [19]:
#collapsing the multiple ATCC names columns into extra rows
new = pd.melt(extra, id_vars=['CF_name1', 'CF_name2', 'CF_alt_name','locus_tag'],
              value_vars=['ATCC_name1', 'ATCC_name2', 'ATCC_name3', 'ATCC_name4'], 
              value_name='ATCC_names')

new = new.dropna(subset=['ATCC_names'])

new.fillna('-', inplace=True)

In [20]:
# making unique ATCC name to prepare to map with full mapping
# combining multiple alphabetical names rows into one row seprated by commas 
CF_final = new.groupby(['ATCC_names']).agg(
    CF_name1=pd.NamedAgg(column='CF_name1', aggfunc=lambda x:','.join(x)), 
    CF_name2=pd.NamedAgg(column='CF_name2', aggfunc=lambda x:','.join(x)), 
    CF_alt=pd.NamedAgg(column='CF_alt_name', aggfunc=lambda x:','.join(x)), 
    locus_tag=pd.NamedAgg(column='locus_tag', aggfunc=lambda x:','.join(x))).reset_index()

In [21]:
# collapse the columns for multiple alphabetical names for both CF gene infos 
extra_names = CF_final['CF_name1'].str.split(',', n=4, expand=True)
extra_names.columns = ['CF_name1', 'CF_name2', 'CF_name3', 'CF_name4']
extra_names2 = CF_final['CF_name2'].str.split(',', n=4, expand=True)
extra_names2.columns = ['CF_name5', 'CF_name6', 'CF_name7', 'CF_name8']

extra = pd.concat([extra_names, extra_names2], axis=1).reset_index(drop=True)

#change df into np array
temp=np.array(extra)
del extra

#remove NA and only keep unique names
fixed =[]
for i in temp: 
     fixed.append(sorted(list(set(i[~pd.isnull(i)])))[::-1])
del temp

#remove '-' names 
fixed = pd.DataFrame(fixed)
fixed = fixed.replace('-', None)
fixed.columns = ['CF_names1', 'CF_names2', 'CF_names3', 'CF_names4', 'CF_names5' ]

In [22]:
#Collapse CF alternative names into columns
alt_names = CF_final['CF_alt'].str.split(',', n=4, expand=True)
alt_names=np.array(alt_names)
alt=[]
for i in alt_names:
    alt.append(sorted(list(set(i[~pd.isnull(i)])))[::-1]) #taking away NAs and Nones and duplicates 
alt = pd.DataFrame(alt)
alt= alt.replace('-', None)
alt.columns = ['CF_alt1', 'CF_alt2', 'CF_alt3', 'CF_alt4']

In [23]:
CF_final = pd.concat([CF_final[['ATCC_names']], fixed, alt], axis=1)
CF_final.head()

,ATCC_names,CF_names1,CF_names2,CF_names3,CF_names4,CF_names5,CF_alt1,CF_alt2,CF_alt3,CF_alt4
0,DR76_RS00090,dnaJ,None,None,None,None,c0020,None,None,None
1,DR76_RS00095,dnaJ,None,None,None,None,c0022,None,None,None
2,DR76_RS00100,yaaY,None,None,None,None,c0028,None,None,None
3,DR76_RS00120,ileS,None,None,None,None,c0030,None,None,None
4,DR76_RS00125,rihC,None,None,None,None,c0035,None,None,None


### Full Mapping 

In [24]:
##Full Mapping--------
full_mapping = pd.read_table('./ATCC_CFT073_K12MG_K12DH_BW_gene_map.txt')
full_mapping.columns = ['ATCC', 'ATCC_names', 'BW', 'BW_names', 'DH', 'DH_names', 'MG', 'MG_names', 'drop']
full_mapping = full_mapping[['ATCC_names','BW_names', 'DH_names', 'MG_names']]
full_mapping = full_mapping.drop_duplicates(keep='first') 
#898

In [25]:
full_mapping.describe()

,ATCC_names,BW_names,DH_names,MG_names
count,898,898,898,898
unique,690,692,686,671
top,DR76_RS03200,BW25113_RS04150,ECDH10B_RS03455,b0054
freq,5,5,5,4


In [26]:
#merge DH names
full_mapping = pd.merge(full_mapping, W3_final, on='DH_names', how='left')
#898

#merge ATCC names
full_mapping = pd.merge(full_mapping, CF_final, on='ATCC_names', how='left') 
full_mapping.drop(['CF_names5'], axis=1, inplace=True)
#898

#merge MG names
MG_info = pd.read_table('../Ecoli_K12_MG1655_chrom_U00096_gene_info.txt')[['gbk_gene_id', 'gbk_locus_tag']]
MG_info.columns=['MG_id', 'MG_names']

full_mapping = pd.merge(full_mapping, MG_info, on='MG_names', how='left')
#898

In [27]:
W3_final.describe()

,DH_names,W3_name1,W3_name2,W3_name3
count,953,953,58,4
unique,953,593,55,4
top,ECDH10B_RS20390,pepD,sufB,quuD
freq,1,11,2,1


In [28]:
CF_final.describe()

,ATCC_names,CF_names1,CF_names2,CF_names3,CF_names4,CF_names5,CF_alt1,CF_alt2,CF_alt3,CF_alt4
count,973,973,307,29,10,1,973,79,8,2
unique,973,709,198,28,10,1,839,77,8,2
top,DR76_RS02110,bet,msrP,yeaQ,garL,mrdA,c2468,c3397,c2200,c4955
freq,1,21,8,2,1,1,6,2,1,1


In [29]:
full_mapping.isna().sum()

ATCC_names      0
BW_names        0
DH_names        0
MG_names        0
W3_name1      713
W3_name2      886
W3_name3      897
CF_names1     729
CF_names2     843
CF_names3     891
CF_names4     891
CF_alt1       729
CF_alt2       878
CF_alt3       891
CF_alt4       896
MG_id           0
dtype: int64

### Comparing to BLAST 

In [30]:
##Comparing---------------
ATCC_DH_blast = pd.read_table('../RHfiles/RBBH_ATCC_DH10B_more_sensitive.tab')
MG_DH_blast = pd.read_table('../RHfiles/RBH_out_MG1655_DH10B_more_sensitive_dmnd_uniq.tab')
MG_ATCC_blast = pd.read_table('../RHfiles/RBH_out_MG1655_ATCC_more_sensitive_dmnd_uniq.tab')

ATCC_DH_blast = ATCC_DH_blast[['A_gene', 'A_alt_name', 'B_gene']]
ATCC_DH_blast.columns = ['ATCC_gene','ATCC_gene2', 'DH_gene']
#3244 rows from blast 

MG_DH_blast = MG_DH_blast[['A_gene', 'A_alt_name', 'B_gene']]
MG_DH_blast.columns = ['MG_gene','MG_gene2', 'DH_gene']
#4162 rows from blast 

MG_ATCC_blast = MG_ATCC_blast[['A_gene', 'A_alt_name', 'B_gene', 'B_alt_name']]
MG_ATCC_blast.columns = ['MG_gene','MG_gene2', 'ATCC_gene', 'ATCC_gene2']
#3264 rows from blast 

In [31]:
#remove redundant ecpD gene
ATCC_DH_blast = ATCC_DH_blast[ATCC_DH_blast['DH_gene']!='yadV']

### ATCC VS DH 

In [32]:
#subsetting from full mapping
ATCC_DH_mapping = full_mapping.drop(['BW_names', 'MG_names','MG_id'], axis=1)

In [33]:
#drop not found DH
ATCC_DH_mapping = ATCC_DH_mapping.dropna(subset=list(ATCC_DH_mapping.columns[2:4]),how='all').drop_duplicates(keep='first')

#drop not found ATCC
ATCC_DH_mapping = ATCC_DH_mapping.dropna(subset=list(ATCC_DH_mapping.columns[5:]),how='all').drop_duplicates(keep='first')

### Cleaning AD blast 
alt names are in the alphabetical names columns if alp name is missing 

In [34]:
## fix shifting in alt gene names 
ATCC_DH_blast['ATCC_genes'] = ATCC_DH_blast['ATCC_gene']
ATCC_DH_blast['ATCC_gene'] = np.where(ATCC_DH_blast['ATCC_gene2'].isna(), np.NaN, ATCC_DH_blast['ATCC_gene'])
ATCC_DH_blast['ATCC_alt2'] = np.where(ATCC_DH_blast['ATCC_genes'].str.startswith('c'), 
                                      ATCC_DH_blast['ATCC_genes'], np.NaN)
ATCC_DH_blast['ATCC_gene2'] = np.where(ATCC_DH_blast['ATCC_gene2'].isna(), ATCC_DH_blast['ATCC_alt2'],
                                      ATCC_DH_blast['ATCC_gene2'])
ATCC_DH_blast = ATCC_DH_blast[['ATCC_gene', 'ATCC_gene2', 'DH_gene']]
ATCC_DH_blast['ATCC_gene2'] = ATCC_DH_blast['ATCC_gene2'].apply(lambda x:str(x))

In [35]:
## comma problems in more than 1 alternative names
ATCC_DH_blast['ATCC_alt'] = ATCC_DH_blast['ATCC_gene2'].apply(lambda x:x.split(',')[0])
alt2 =[]
for i in ATCC_DH_blast['ATCC_gene2']:
    try: 
        alt2.append(i.split(',')[1])
    except (IndexError):
        alt2.append('NA')

ATCC_DH_blast['ATCC_alt2'] = alt2

In [36]:
#extra rows
temp = ATCC_DH_blast[ATCC_DH_blast['ATCC_alt2'] != 'NA'].reset_index(drop=True)
temp.tail()

,ATCC_gene,ATCC_gene2,DH_gene,ATCC_alt,ATCC_alt2
6,ecpD,"c0401, c0171",ecpD,c0401,c0171
7,sucC,"c0805, c5036",sucC,c0805,c5036
8,tdcD,"c3873, c4530",tdcD,c3873,c4530
9,rbsK,"c4680, c0331, c2492",rbsK,c4680,c0331
10,arcB,"c5348, c3970",arcB,c5348,c3970


In [37]:
temp = temp[['ATCC_gene','ATCC_alt2','DH_gene']]
temp.rename(columns={'ATCC_alt2':'ATCC_alt'},inplace=True)

temp = temp.append(pd.DataFrame({'ATCC_gene':['rbsK'],'ATCC_alt':['c2492'],
                                 'DH_gene':['rbsK']}), ignore_index=True)

ATCC_DH_blast = ATCC_DH_blast[['ATCC_gene','ATCC_alt','DH_gene']]
ATCC_DH_blast = pd.concat([ATCC_DH_blast,temp]).reset_index(drop=True)

#the row that was accidentally filter bc of true NA in alt name
ATCC_DH_blast = ATCC_DH_blast.drop([1396]).reset_index(drop=True)

ATCC_DH_blast.describe()

,ATCC_gene,ATCC_alt,DH_gene
count,3152,3254,3254
unique,3140,3254,3241
top,rbsK,c0331,rbsK
freq,3,1,3


In [38]:
ATCC_DH_mapping.describe()

,ATCC_names,DH_names,W3_name1,W3_name2,W3_name3,CF_names1,CF_names2,CF_names3,CF_names4,CF_alt1,CF_alt2,CF_alt3,CF_alt4
count,38,38,38,2,1,38,17,6,6,38,10,5,1
unique,33,35,35,2,1,30,12,3,3,31,7,2,1
top,DR76_RS00825,ECDH10B_RS06710,setA,truD,cysC,yadB,sfsA,gluQRS,dksA,c0179,c0178,c0177,c0164
freq,4,2,2,1,1,4,4,4,4,4,4,4,1


In [39]:
ATCC_DH_blast[ATCC_DH_blast['ATCC_gene']=='acpS']

,ATCC_gene,ATCC_alt,DH_gene
127,acpS,c3087,acpS


### Comparing

In [40]:
AD_blast = np.array(ATCC_DH_blast[['ATCC_gene','DH_gene']])
AD_blast_alt = np.array(ATCC_DH_blast[['ATCC_alt', 'DH_gene']])

In [41]:
#checking blast using alphabet names only
AD_blast_check = pd.DataFrame({})
for i in range(len(AD_blast)):
    df = ATCC_DH_mapping.iloc[:,:9][((ATCC_DH_mapping['CF_names1']== AD_blast[i][0])|
               (ATCC_DH_mapping['CF_names2']==AD_blast[i][0])|
               (ATCC_DH_mapping['CF_names3']==AD_blast[i][0])|
               (ATCC_DH_mapping['CF_names4']==AD_blast[i][0]))]
    if len(df)>=1:
        df['matched_CF']= AD_blast[i][0]
        df['matched_W3']= AD_blast[i][1]
        df['check'] = ((ATCC_DH_mapping['W3_name1']==AD_blast[i][1])|
                (ATCC_DH_mapping['W3_name2']==AD_blast[i][1])|
               (ATCC_DH_mapping['W3_name3']==AD_blast[i][1]))
        AD_blast_check = AD_blast_check.append(df, ignore_index=True)

In [42]:
#checking blast_alt using alt names only
AD_blast_alt_check = pd.DataFrame({})
for i in range(len(AD_blast_alt)):
    df = ATCC_DH_mapping.drop(ATCC_DH_mapping.columns[5:9], axis=1)[
        ((ATCC_DH_mapping['CF_alt1']== AD_blast_alt[i][0])|
               (ATCC_DH_mapping['CF_alt2']==AD_blast_alt[i][0])|
               (ATCC_DH_mapping['CF_alt3']==AD_blast_alt[i][0])|
               (ATCC_DH_mapping['CF_alt4']==AD_blast_alt[i][0]))]
    if len(df)>=1:
        df['matched_CF']= AD_blast_alt[i][0]
        df['matched_W3']= AD_blast_alt[i][1]
        df['check'] = ((ATCC_DH_mapping['W3_name1']==AD_blast_alt[i][1])|
                (ATCC_DH_mapping['W3_name2']==AD_blast_alt[i][1])|
               (ATCC_DH_mapping['W3_name3']==AD_blast_alt[i][1]))
        AD_blast_alt_check = AD_blast_alt_check.append(df, ignore_index=True)

### MG VS DH

In [65]:
#subsetting from full mapping
MG_DH_mapping = full_mapping.drop(full_mapping.columns[7:-1].to_list(), axis=1).drop(full_mapping.columns[0:2].to_list(), axis=1)

In [71]:
#drop not found DH - no need to drop MG b/c MG is very complete
MG_DH_mapping = MG_DH_mapping.dropna(subset=list(MG_DH_mapping.columns[2:5]),how='all').drop_duplicates(keep='first')

In [73]:
MG_DH_mapping.head()

,DH_names,MG_names,W3_name1,W3_name2,W3_name3,MG_id
4,ECDH10B_RS20020,b4483,nikB,None,None,tatD
8,ECDH10B_RS20240,b3879,glgX,None,None,yihR
12,ECDH10B_RS20280,b3888,glpE,None,None,yiiD
15,ECDH10B_RS20335,b3897,malP,malQ,None,frvR
22,ECDH10B_RS20605,b3949,nirC,None,None,frwC


### Comparing

In [82]:
## Load blast files
MG_DH_blast = pd.read_table('../RHfiles/RBH_out_MG1655_DH10B_more_sensitive_dmnd_uniq.tab')

MG_DH_blast = MG_DH_blast[['A_gene', 'A_alt_name', 'B_gene']]
MG_DH_blast.columns = ['MG_gene','MG_alt', 'DH_gene']

In [84]:
MD_blast = np.array(MG_DH_blast[['MG_gene','DH_gene']])
MD_blast_alt = np.array(MG_DH_blast[['MG_alt', 'DH_gene']])

In [93]:
MG_DH_blast.head()

,MG_gene,MG_alt,DH_gene
0,yciZ,b4596,yciZ
1,yncL,b4598,yncL
2,mgtS,b4599,mgtS
3,ydgU,b4601,ydgU
4,ynhF,b4602,ynhF


In [90]:
#checking blast using alphabet names only
MD_blast_check = pd.DataFrame({})
for i in range(len(MD_blast)):
    df = MG_DH_mapping[(MG_DH_mapping['MG_id']== MD_blast[i][0])]
    if len(df)>=1:
        df['matched_MG']= MD_blast[i][0]
        df['matched_DH']= MD_blast[i][1]
        df['check'] = ((MG_DH_mapping['W3_name1']==MD_blast[i][1])|
                (MG_DH_mapping['W3_name2']==MD_blast[i][1])|
               (MG_DH_mapping['W3_name3']==MD_blast[i][1]))
        MD_blast_check = MD_blast_check.append(df, ignore_index=True)

In [92]:
print(len(MD_blast_check), MD_blast_check['check'].sum())
MD_blast_check

146 25


,DH_names,MG_names,W3_name1,W3_name2,W3_name3,MG_id,matched_MG,matched_DH,check
0,ECDH10B_RS06710,b4596,yciH,None,None,yciZ,yciZ,yciZ,False
1,ECDH10B_RS17780,b3417,fdnG,None,None,malP,malP,malP,False
2,ECDH10B_RS00755,b0149,mrcB,None,None,mrcB,mrcB,mrcB,True
3,ECDH10B_RS01135,b0215,metQ,None,None,dnaQ,dnaQ,dnaQ,False
4,ECDH10B_RS14875,b2837,lysA,None,None,galR,galR,galR,False
...,...,...,...,...,...,...,...,...,...
141,ECDH10B_RS14610,b2789,rlmD,None,None,gudP,gudP,gudP,False
142,ECDH10B_RS13355,b2536,trmJ,hcaR,None,hcaT,hcaT,hcaT,False
143,ECDH10B_RS01205,b0223,lpcA,None,None,yafJ,yafJ,yafJ,False
144,ECDH10B_RS01225,b0227,lpcA,None,None,yafL,yafL,yafL,False


In [94]:
#checking blast_alt using alt names only
MD_blast_alt_check = pd.DataFrame({})
for i in range(len(MD_blast_alt)):
    df = MG_DH_mapping[(MG_DH_mapping['MG_names']== MD_blast_alt[i][0])]
    if len(df)>=1:
        df['matched_MG']= MD_blast_alt[i][0]
        df['matched_W3']= MD_blast_alt[i][1]
        df['check'] = ((MG_DH_mapping['W3_name1']==MD_blast_alt[i][1])|
                (MG_DH_mapping['W3_name2']==MD_blast_alt[i][1])|
               (MG_DH_mapping['W3_name3']==MD_blast_alt[i][1]))
        MD_blast_alt_check = MD_blast_alt_check.append(df, ignore_index=True)

In [96]:
print(len(MD_blast_alt_check), MD_blast_alt_check['check'].sum())
MD_blast_check

154 26


,DH_names,MG_names,W3_name1,W3_name2,W3_name3,MG_id,matched_MG,matched_DH,check
0,ECDH10B_RS06710,b4596,yciH,None,None,yciZ,yciZ,yciZ,False
1,ECDH10B_RS17780,b3417,fdnG,None,None,malP,malP,malP,False
2,ECDH10B_RS00755,b0149,mrcB,None,None,mrcB,mrcB,mrcB,True
3,ECDH10B_RS01135,b0215,metQ,None,None,dnaQ,dnaQ,dnaQ,False
4,ECDH10B_RS14875,b2837,lysA,None,None,galR,galR,galR,False
...,...,...,...,...,...,...,...,...,...
141,ECDH10B_RS14610,b2789,rlmD,None,None,gudP,gudP,gudP,False
142,ECDH10B_RS13355,b2536,trmJ,hcaR,None,hcaT,hcaT,hcaT,False
143,ECDH10B_RS01205,b0223,lpcA,None,None,yafJ,yafJ,yafJ,False
144,ECDH10B_RS01225,b0227,lpcA,None,None,yafL,yafL,yafL,False


## MG and ATCC

In [42]:
MG_ATCC_blast = pd.read_table('../RHfiles/RBH_out_MG1655_ATCC_more_sensitive_dmnd_uniq.tab')
MG_ATCC_blast = MG_ATCC_blast[['A_gene', 'A_alt_name', 'B_gene', 'B_alt_name']]
MG_ATCC_blast.columns = ['MG_gene','MG_gene2', 'ATCC_gene', 'ATCC_gene2']
#3264 rows from blast 

#remove redundant ecpD gene
MG_ATCC_blast = MG_ATCC_blast[MG_ATCC_blast['MG_gene']!='yadV']

In [43]:
## Cleaning ATCC blast
MG_ATCC_blast.columns = ['MG_gene','MG_gene2', 'ATCC_gene', 'ATCC_gene2']

In [44]:
## fix shifting in alt gene names 
MG_ATCC_blast['ATCC_genes'] = MG_ATCC_blast['ATCC_gene']
MG_ATCC_blast['ATCC_gene'] = np.where(MG_ATCC_blast['ATCC_gene2'].isna(), np.NaN, MG_ATCC_blast['ATCC_gene'])


MG_ATCC_blast['ATCC_alt2'] = np.where(MG_ATCC_blast['ATCC_genes'].str.startswith('c'), 
                                      MG_ATCC_blast['ATCC_genes'], np.NaN)

MG_ATCC_blast['ATCC_gene2'] = np.where(MG_ATCC_blast['ATCC_gene2'].isna(), MG_ATCC_blast['ATCC_alt2'],
                                      MG_ATCC_blast['ATCC_gene2'])

MG_ATCC_blast = MG_ATCC_blast[['MG_gene', 'MG_gene2','ATCC_gene', 'ATCC_gene2']]

MG_ATCC_blast['ATCC_gene2'] = MG_ATCC_blast['ATCC_gene2'].apply(lambda x:str(x))

In [45]:
## comma problems in more than 1 alternative names
MG_ATCC_blast['ATCC_alt'] = MG_ATCC_blast['ATCC_gene2'].apply(lambda x:x.split(',')[0])


alt2 =[]
for i in MG_ATCC_blast['ATCC_gene2']:
    try: 
        alt2.append(i.split(',')[1])
    except (IndexError):
        alt2.append('NA')

MG_ATCC_blast['ATCC_alt2'] = alt2

In [46]:
#extra rows
temp = MG_ATCC_blast[MG_ATCC_blast['ATCC_alt2'] != 'NA'].reset_index(drop=True)

In [47]:
temp = temp[['MG_gene', 'MG_gene2','ATCC_gene','ATCC_alt2']]
temp.rename(columns={'ATCC_alt2':'ATCC_alt'},inplace=True)

temp = temp.append(pd.DataFrame({'MG_gene':['rbsK'],'MG_gene2':['b3752'],
                                 'ATCC_gene':['rbsK'],'ATCC_alt':['c2492']}), ignore_index=True)

MG_ATCC_blast = MG_ATCC_blast[['MG_gene', 'MG_gene2','ATCC_gene','ATCC_alt']]
MG_ATCC_blast = pd.concat([MG_ATCC_blast,temp]).reset_index(drop=True)

#the row that was accidentally filter bc of true NA in alt name
#MG_ATCC_blast = MG_ATCC_blast.drop([1396]).reset_index(drop=True)

MG_ATCC_blast.describe()

,MG_gene,MG_gene2,ATCC_gene,ATCC_alt
count,3275,3273,3163,3275
unique,3263,3260,3149,3264
top,rbsK,b3752,rbsK,c3611
freq,3,3,3,6


In [48]:
##comparing
MG_ATCC_mapping = full_mapping[['MG_gbk_name', 'MG_gbk_name2', 'CF_name1',
'CF_name2', 'CF_alt_name']].drop_duplicates(keep='first')
#3558
MG_ATCC_mapping.describe()

,MG_gbk_name,MG_gbk_name2,CF_name1,CF_name2,CF_alt_name
count,4879,4879,2357,2725,3694
unique,3501,3503,1488,1809,2487
top,flu,b2000,tnpA,fhuE,c2459
freq,9,9,12,7,8


In [49]:
MA_comp = pd.merge(MG_ATCC_blast, MG_ATCC_mapping, left_on='ATCC_alt', right_on='CF_alt_name',
                  how='left')
MA_non_match = MA_comp[MA_comp['CF_alt_name'].isna()]

In [50]:
MA_comp.describe()

,MG_gene,MG_gene2,ATCC_gene,ATCC_alt,MG_gbk_name,MG_gbk_name2,CF_name1,CF_name2,CF_alt_name
count,4121,4119,3978,4121,2512,2512,1980,2376,2512
unique,3263,3260,3149,3264,1762,1763,1281,1569,1664
top,fhuE,b1102,fhuE,c1374,flu,b2000,ptsP,fhuE,c1374
freq,7,7,7,7,8,8,8,7,7


In [51]:
MA_comp.dropna(subset=['CF_alt_name'], inplace=True)
MA_comp['check1'] = (MA_comp['MG_gene'] == MA_comp['MG_gbk_name']) | (MA_comp['MG_gene2'] == MA_comp['MG_gbk_name2'])
print(len(MA_comp), MA_comp['check1'].sum())

2512 0


In [52]:
#  MA_comp[(MA_comp['check1']==False) & (MA_comp['MG_gene2'].str.len()==4)]
# #filter less than 4 digits gene names and send screenshot

In [53]:
MA_non_match = MA_non_match[['MG_gene','MG_gene2','ATCC_gene']].drop_duplicates(keep='first')

In [54]:
MG_ATCC_mapping.head()

,MG_gbk_name,MG_gbk_name2,CF_name1,CF_name2,CF_alt_name
0,yigI,b3820,ybeY,ybeY,c0744
1,yigI,b3820,NaN,ybeZ,c0745
2,pldA,b3821,corC,ybeX,c0743
3,recQ,b3822,NaN,NaN,c0741
4,rhtC,b3823,NaN,NaN,c0741


In [55]:
MA_non_match.describe()

,MG_gene,MG_gene2,ATCC_gene
count,1601,1599,1549
unique,1601,1599,1549
top,ytiA,b2297,yhfS
freq,1,1,1


In [56]:
MG_ATCC_mapping.drop_duplicates(keep='first').describe()

,MG_gbk_name,MG_gbk_name2,CF_name1,CF_name2,CF_alt_name
count,4879,4879,2357,2725,3694
unique,3501,3503,1488,1809,2487
top,flu,b2000,tnpA,fhuE,c2459
freq,9,9,12,7,8


In [57]:
MA_comp2 = pd.merge(MA_non_match, MG_ATCC_mapping[['MG_gbk_name','MG_gbk_name2', 'CF_name1', 'CF_name2']],
                    left_on='MG_gene2', right_on='MG_gbk_name2')

In [58]:
print(len(MA_comp2))
MA_comp2.describe()

2015


,MG_gene,MG_gene2,ATCC_gene,MG_gbk_name,MG_gbk_name2,CF_name1,CF_name2
count,2015,2015,1959,2015,2015,1022,1158
unique,1458,1458,1425,1458,1458,754,888
top,yigE,b4482,ygbN,yigE,b4482,clbJ,fhuE
freq,7,7,5,7,7,7,4


In [64]:
MA_comp2 = MA_comp2.dropna(how='all', subset=['CF_name1','CF_name2'])
len(MA_comp2)

1262

In [65]:
MA_comp2['check1'] = (MA_comp2['ATCC_gene'] == MA_comp2['CF_name1']) | (MA_comp2['ATCC_gene'] == MA_comp2['CF_name2'])

In [66]:
MA_comp2[MA_comp2['check1']==False]

,MG_gene,MG_gene2,ATCC_gene,MG_gbk_name,MG_gbk_name2,CF_name1,CF_name2,check1
0,yciY,b4595,yciY,yciY,b4595,arcB,arcB,False
3,frdA,b4154,frdA,frdA,b4154,hisG,hisG,False
4,frdA,b4154,frdA,frdA,b4154,hisD,hisD,False
5,frdA,b4154,frdA,frdA,b4154,hisC,hisC,False
6,gdhA,b1761,gdhA,gdhA,b1761,NaN,yqeG,False
...,...,...,...,...,...,...,...,...
2002,quuD,b0551,ybcQ,quuD,b0551,NaN,yhfK,False
2004,tauC,b0367,tauC,tauC,b0367,ssb1,ssb,False
2006,copA,b0484,ybaR,copA,b0484,rrf,rrfB,False
2007,copA,b0484,ybaR,copA,b0484,murB,murB,False


In [ ]:
#%%
table = pd.DataFrame({'Strains':['AD_gbk','AD_blast','MD_gbk','MD_blast','MA_gbk','MA_blast'],
'rows':[len(ATCC_DH_mapping1), len(ATCC_DH_blast),len(MG_DH_mapping1), len(MG_DH_blast),
len(MG_ATCC_mapping1), len(MG_ATCC_blast)],
'mutual':[len(AD_comp)+len(AD_comp2),len(AD_comp)+len(AD_comp2),
len(MD_comp)+len(MD_comp2),len(MD_comp)+len(MD_comp2),
len(MA_comp),len(MA_comp)],
'matched':[(AD_comp['check'].sum()+AD_comp2['check'].sum()),
(AD_comp['check'].sum()+AD_comp2['check'].sum()),
(MD_comp['check'].sum()+MD_comp2['check'].sum()),
(MD_comp['check'].sum()+MD_comp2['check'].sum()),
MA_comp['check'].sum(),
MA_comp['check'].sum()]})

table['mismatch'] = table['mutual'] - table['matched']

table

In [17]:
# %%
#extracting genes and comparing them 
AD_final = pd.concat([AD_comp[AD_comp['check']], AD_comp2[AD_comp2['check']]])
AD_final = AD_final.iloc[:, 0:3]

MD_final = pd.concat([MD_comp[MD_comp['check']], MD_comp2[MD_comp2['check']]])
MD_final = MD_final.iloc[:, 0:3]

MA_final = MA_comp[MA_comp['check']].iloc[:, [0,1,3]]

#%%
#merging MG 

MG_merge = pd.merge(MD_final, MA_final, on='MG_gene')
##checking if both bxxx MG names are the same, 
##use MG_gbk_name2 bc it is more complete (1 less NAs)
#MG_merge['check'] = MG_merge['MG_gbk_name2'] == MG_merge['MG_gene2']
#MG_merge[~MG_merge['check']] #1 row of NA 

MG_merge = MG_merge.iloc[:, [0,2,3,4]]
#1366 



In [18]:
MD_final['check2'] = MD_final['MG_gene'].apply(lambda x: x in MA_final['MG_gene'].to_list())
MD_final[~MD_final['check2']]

,MG_gene,MG_gene2,DH_gene,check2
1,yncL,b4598,yncL,False
4,ynhF,b4602,ynhF,False
5,ypfM,b4606,ypfM,False
13,yoaK,b4676,yoaK,False
14,yoaJ,b4675,yoaJ,False
...,...,...,...,...
92,fumE,b2929,fumE,False
93,hemW,b2955,hemW,False
94,trmB,b2960,trmB,False
96,rapZ,b3205,rapZ,False


In [ ]:
#keep the NA genes in ATCC and DH 

In [26]:
full_mapping[full_mapping['MG_gbk_name']=='yncL']

,ATCC,ATCC_names,BW,BW_names,DH,DH_names,MG,MG_names,ATCC_gbk_name2,ATCC_gbk_name,DH_gbk_name,MG_gbk_name,MG_gbk_name2
1462,CP009072,DR76_RS27345,CP009273,BW25113_RS23775,NC_010473,ECDH10B_RS24100,U00096000,b4598,NaN,NaN,yncL,yncL,b4598


In [23]:
# %%
### why there are missing genes


# %%
all_merge = pd.merge(MG_merge, AD_final, on='DH_gene')
#1366
# all_merge['check'] = all_merge['ATCC_gene_x'] == all_merge['ATCC_gene_y']
# len(all_merge) - all_merge['check'].sum() #0 rows 
# retains all DH genes, all ATCC genes are consitent

# %%
all_merge_check = all_merge[['MG_gbk_name2']] #MG has the most unique gene names 
#all_merge['check']=all_merge['MG_gene']==all_merge['DH_gene']
#all_merge['check'].sum() #1366 all DH genes were the same as MG genes 

1366

In [22]:
all_merge.head()

,MG_gene,DH_gene,ATCC_gene_x,MG_gbk_name2,ATCC_gene_y,ATCC_gene2
0,frdA,frdA,frdA,b4154,frdA,c5242
1,gdhA,gdhA,gdhA,b1761,gdhA,c2162
2,nrdA,nrdA,nrdA,b2234,nrdA,c2776
3,aspC,aspC,aspC,b0928,aspC,c1070
4,phoA,phoA,phoA,b0383,phoA,c0490


In [32]:
full_mapping[full_mapping['ATCC_names']=='DR76_RS03755']

,ATCC,ATCC_names,BW,BW_names,DH,DH_names,MG,MG_names,ATCC_gbk_name2,ATCC_gbk_name,DH_gbk_name,MG_gbk_name,MG_gbk_name2
3647,CP009072,DR76_RS03755,CP009273,BW25113_RS19455,NC_010473,ECDH10B_RS20010,U00096000,b3743,c4671,asnC,NaN,asnC,b3743


In [ ]:
## add extra columns 

In [11]:
final_map = pd.merge(full_mapping, all_merge_check, left_on='MG_names',
right_on='MG_gbk_name2') #mapped back to full mappings
final_map = final_map.iloc[:, [1,3,5,7,11]]
final_map.drop_duplicates(subset=['MG_names'], inplace=True)
# %%
final_map.to_csv('final_genes.csv', index=None)

In [12]:
final_map.describe()

,ATCC_names,BW_names,DH_names,MG_names,MG_gbk_name
count,1366,1366,1366,1366,1366
unique,1366,1366,1366,1366,1366
top,DR76_RS11990,BW25113_RS16635,ECDH10B_RS21720,b1883,amiA
freq,1,1,1,1,1
